In [22]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from filter_data import LinePlots


pd.options.mode.chained_assignment = None


Alle data import

In [20]:
file_path = 'TILL6022_Emission_Dataset.csv'
df = pd.read_csv(file_path, delimiter=',')

# filter dataset on Transport sectors: 'International Shipping' or 'International Aviation' or 'Domestic Aviation' or 'Ground Transport'
sectors = ['International Shipping', 'International Aviation', 'Domestic Aviation', 'Ground Transport']
Transport_Sector =  df[df.sector.isin(sectors)]

# World is not an country, so we filter World out of the data set. 
# We then grouped by country and date to sum the different sectors into a total sum per country per day co2

Transport_not_World = Transport_Sector[Transport_Sector.country != 'WORLD'].groupby(['country', 'date']).sum()

# Reset index for optimal dataset and split date into years to access the data more easily in later steps
# As we want to plot the year on each other, we converted every year to the same year. In the case 2019

Transport_Total = Transport_not_World.reset_index(['country', 'date'])
Transport_Total[["year", "month", "day"]] = Transport_Total["date"].str.split("-", expand = True)
Transport_Total['Date2'] = '2019' + '-' + Transport_Total['month'].astype(str) + '-' + Transport_Total['day']

# We did the same for the other dataset

Transport_Sector[["year", "month", "day"]] = Transport_Sector["date"].str.split("-", expand = True)
Transport_Sector['Date2'] = '2019' + '-' + Transport_Sector['month'].astype(str) + '-' + Transport_Sector['day']

Years = ['2019', '2020', '2021']
Countries = ['UK', 'US', 'China', 'EU27 & UK', 'France', 'Germany', 'India', 'Italy', 'Japan', 'Russia', 'Spain']
sectors = ['International Aviation', 'Domestic Aviation', 'Ground Transport']




Create figure met lines

In [56]:
fig = make_subplots(rows = 4, 
                    cols = 1,
                    subplot_titles = ('Total', 'International Aviation', 'Domestic Aviation', 'Ground Transport'))

# add traces for every combination of country, sector and year.

for k in Countries:
    for i in Years:
        z = Transport_Total[(Transport_Total.country == k) & (Transport_Total.year == i)]
        Int_Av = Transport_Sector[(Transport_Sector.country == k) & (Transport_Sector.year == i) & (Transport_Sector.sector == 'International Aviation')]
        Do_Av = Transport_Sector[(Transport_Sector.country == k) & (Transport_Sector.year == i) & (Transport_Sector.sector == 'Domestic Aviation')]
        Gr_Trans = Transport_Sector[(Transport_Sector.country == k) & (Transport_Sector.year == i) & (Transport_Sector.sector == 'Ground Transport')]
        lop = fig.add_traces(
                    [
                    go.Scatter(x=list(z.Date2),
                        y=list(z.co2),
                        name= k + ' ' + i + ' ' + 'Total',
                        visible = True,
                        legendgroup = 1),        
                    go.Scatter(x=list(Int_Av.Date2),
                            y=list(Int_Av.co2),
                            name= k + ' ' + i + ' ' + 'International Aviation',
                            visible = True,
                            legendgroup = 2),
                    go.Scatter(x=list(Do_Av.Date2),
                            y=list(Do_Av.co2),
                            name= k + ' ' + i + ' ' + 'Domestic Aviation',
                            visible = True,
                            legendgroup = 3),
                    go.Scatter(x=list(Gr_Trans.Date2),
                            y=list(Gr_Trans.co2),
                            name= k + ' ' + i + ' ' + 'Ground Transport',
                            visible = True,
                            legendgroup = 4)]
                , rows=list(range(1,5)), cols=[1,1,1,1]
                )

# create dropdown menu

list_updatemenus = []

for n, country in enumerate(Countries):
    visible = 4*[False] * len(Years) * len(Countries)
    try:
        for x in range(4*len(Years)):
            visible[4*len(Years)*n + x] = True
    except IndexError:
        continue
    temp_dict = dict(label = str(country),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': ('CO2 emissions comparison 2019 versus 2020 by sector in %s'% country)}])
    list_updatemenus.append(temp_dict)

fig.update_layout(updatemenus=list([dict(buttons= list_updatemenus)]))

# location of dropdown menu on plot

fig.update_layout(
    updatemenus=[
        dict(
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.04,
            yanchor="top"
        ),
    ]
)

# add height, title, labels etc. to figure

fig.update_layout(
    height = 1600,
    title_text = 'CO2 emissions comparison 2019 versus 2020',
    
    xaxis1_title = 'Date',
    xaxis2_title = 'Date',
    xaxis3_title = 'Date',
    xaxis4_title = 'Date',
    yaxis1_title = 'CO2 emissions x 1000 (ppm)',
    yaxis2_title = 'CO2 emissions x 1000 (ppm)',
    yaxis3_title = 'CO2 emissions x 1000 (ppm)',
    yaxis4_title = 'CO2 emissions x 1000 (ppm)',
    
    xaxis=dict(tickformat="%d-%B"),
                 xaxis2=dict(tickformat="%d-%B"),
                 xaxis3=dict(tickformat="%d-%B"),
                 xaxis4=dict(tickformat="%d-%B"),
    hovermode = 'x unified',
    legend_tracegroupgap = 340,
    )

# show figure
fig.show()


In [16]:
from statistics import mean
Transport_not_World = Transport_Sector[Transport_Sector.country != 'WORLD'].groupby(['country', 'date']).sum()


#reset index for optimal dataset and split date into years

Transport_Final = Transport_not_World.reset_index(['country', 'date'])
Transport_Final[["year", "month", "day"]] = Transport_Final["date"].str.split("-", expand = True)

# create dataframe for sector

Transport_Sector[["year", "month", "day"]] = Transport_Sector["date"].str.split("-", expand = True)
Transport_Sector['Date2'] = '2019' + '-' + Transport_Sector['month'].astype(str) + '-' + Transport_Sector['day']
Transport_Sector2 = Transport_Sector[(Transport_Sector.year == '2019') | (Transport_Sector.year == '2020')]

Transport_not_World = Transport_Sector[Transport_Sector.country != 'WORLD'].groupby(['country', 'date']).sum()

#reset index for optimal dataset and split date into years

Transport_Final = Transport_not_World.reset_index(['country', 'date'])
Transport_Final[["year", "month", "day"]] = Transport_Final["date"].str.split("-", expand = True)
Transport_Final['Date2'] = '2019' + '-' + Transport_Final['month'].astype(str) + '-' + Transport_Final['day']

# add trace
Years = ['2019', '2020']
Countries = ['UK', 'US', 'China', 'EU27 & UK', 'France', 'Germany', 'India', 'Italy', 'Japan', 'Russia', 'Spain']
sectors = ['International Aviation', 'Domestic Aviation', 'Ground Transport']


Delta_Country = []
Date_country = []
Date_country_max = []
Percentage_Delta = []
for k in Countries:
    # for i in Years:
    Delta_Total = Transport_Final[(Transport_Final.country == k) & (Transport_Final.year == '2020')]
    diff_list = []
    for d in range(len(Delta_Total)-1):
            Diff = abs(Delta_Total['co2'].iloc[d] - Delta_Total['co2'].iloc[d+1])
            Co2Value = Delta_Total['co2'].iloc[d-1]
            diff_list.append(Diff)
            Date_country.append(Delta_Total.iloc[d]['date'])
    Max_Diff = max(diff_list)
    percent_Delta = ("%g" % round((Max_Diff/Co2Value)*100,1))
    # print(Meanvalue)
    # print('This is the max diff:' , Max_Diff, k)
    index_maxvalue = diff_list.index(Max_Diff)
    # print("Date at which delta is max: ", Date_country[index_maxvalue])
    maxDATE = Date_country[index_maxvalue]
    Delta_Country.append(Max_Diff)
    Date_country_max.append(maxDATE)
    Percentage_Delta.append(percent_Delta)
# print(Delta_Country)

 
Delta_Country_Do = []
Date_country_Do = []
Date_country_max_Do = []
Percentage_Delta_Do = []
for k in Countries:
    # for i in Years:
    Do_Av = Transport_Sector2[(Transport_Sector2.country == k) & (Transport_Sector2.year == '2020') & (Transport_Sector2.sector == 'Domestic Aviation')]
    diff_list_Do = []
    for d in range(len(Do_Av)-1):
            Diff = abs(Do_Av['co2'].iloc[d] - Do_Av['co2'].iloc[d+1])
            Co2Value = Do_Av['co2'].iloc[d-1]
            diff_list_Do.append(Diff)
            Date_country_Do.append(Do_Av.iloc[d]['date'])
    Max_Diff_Do = max(diff_list_Do)
    percent_Delta_Do = round((Max_Diff_Do/Co2Value)*100,1)
    # print('This is the max diff:' , Max_Diff_Do, k)
    index_maxvalue_Do = diff_list_Do.index(Max_Diff_Do)
    maxDATE_Do = Date_country_Do[index_maxvalue_Do]
    Delta_Country_Do.append(Max_Diff_Do)
    Date_country_max_Do.append(maxDATE_Do)
    Percentage_Delta_Do.append(percent_Delta_Do)
# print(Delta_Country)

Delta_Country_Trans = []
Date_country_Trans = []
Date_country_max_Trans = []
Percentage_Delta_Trans = []
for k in Countries:
    # for i in Years:
    Gr_Trans = Transport_Sector2[(Transport_Sector2.country == k) & (Transport_Sector2.year == '2020') & (Transport_Sector2.sector == 'Ground Transport')]
    diff_list_Trans = []
    for d in range(len(Gr_Trans)-1):
            Diff = abs(Gr_Trans['co2'].iloc[d] - Gr_Trans['co2'].iloc[d+1])
            Co2Value_Trans = Gr_Trans['co2'].iloc[d-1]
            diff_list_Trans.append(Diff)
            Date_country_Trans.append(Gr_Trans.iloc[d]['date'])
    Max_Diff_Trans = max(diff_list_Trans)
    Percent_Delta_Trans = round((Max_Diff_Trans/Co2Value_Trans)*100, 1)
    # print('This is the max diff:' , Max_Diff_Trans, k)
    Delta_Country_Trans.append(Max_Diff_Trans)
    index_maxvalue_Trans = diff_list_Trans.index(Max_Diff_Trans)
    maxDATE_Trans = Date_country_Trans[index_maxvalue_Trans]
    Date_country_max_Trans.append(maxDATE_Trans)
    Percentage_Delta_Trans.append(Percent_Delta_Trans)
# print(Delta_Country)


Delta_Country_Int = []
Date_country_Int = []
Date_country_max_Int = []
Percentage_Delta_Int = []
for k in Countries:
    # for i in Years:
    Int_Av = Transport_Sector2[(Transport_Sector2.country == k) & (Transport_Sector2.year == '2020') & (Transport_Sector2.sector == 'International Aviation')]
    diff_list_Int = []
    for d in range(len(Int_Av)-1):
            Diff = abs(Int_Av['co2'].iloc[d] - Int_Av['co2'].iloc[d+1])
            Meanvalue_Int = mean(Int_Av['co2'])
            diff_list_Int.append(Diff)
            Date_country_Int.append(Int_Av.iloc[d]['date'])
    Max_Diff_Int = max(diff_list_Int)
    Percent_Delta_Int = round((Max_Diff_Int/Meanvalue_Int)*100, 1)
    Delta_Country_Int.append(Max_Diff_Int)
    index_maxvalue_Int = diff_list_Int.index(Max_Diff_Int)
    maxDATE_Int = Date_country_Int[index_maxvalue_Int]
    Date_country_max_Int.append(maxDATE_Int)
    Percentage_Delta_Int.append(Percent_Delta_Int)
# print(Delta_Country)

DeltaTable = pd.DataFrame(
    {'Country': Countries,
     'Biggest Delta 2020': Delta_Country,
     'Percentage biggest Delta 2020': Percentage_Delta,
     'Date biggest Delta 2020': Date_country_max,
     'Biggest Ground Transportation Delta 2020': Delta_Country_Trans,
     'Percentage biggest Ground Transportation Delta 2020': Percentage_Delta_Trans,
     'Date biggest Ground Transportation Delta 2020': Date_country_max_Trans,
     'Biggest Domestic Aviation Delta 2020': Delta_Country_Do,
     'Percentage biggest Domestic Aviation Delta 2020': Percentage_Delta_Do,
     'Date biggest Domestic Aviation Delta 2020': Date_country_max_Do,
     'Biggest International Aviation Delta 2020': Delta_Country_Int,
     'Percentage biggest International Aviation Delta 2020': Percentage_Delta_Int,
     'Date biggest International Aviation Delta 2020': Date_country_max_Int
    })

DeltaTable

import numpy as np 
import pandas as pd



DeltaTable.style.background_gradient(cmap='Blues')
# DeltaTable